<a href="https://colab.research.google.com/github/neiltheblue/loial/blob/main/Loial_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Loial pacakge needs to be available, as package or from the git repo.

In [1]:
!pip install -i https://test.pypi.org/simple/ loial

Looking in indexes: https://test.pypi.org/simple/


# Simple Python builder example

The current Loial project supports two buildedrs. One is the python builder that is used mainly to exercise the framework, and the CC compiler that is used to integrate compiles libraries. So far this has only been tested using Linux.

This first exmaple shows how a python method can be replaced by an alternative implemention.

In [2]:
def grow(x):
  return x + x

assert grow(3) == 6

The alternative implementation. Here the content of the build decorator is used instead of the python method body.

In [3]:
from loial import build

@build('return x ** 2')
def grow(x):
  return x + x

assert grow(3) == 9

This alternative behaviour can be turned off.

In [4]:
from loial import build

@build('return x ** 2', replace=False)
def grow(x):
  return x + x

assert grow(3) == 6

If the alternative code does not compile, then the original body will not be replaced,

In [9]:
from loial import build

@build('return x to the power of 2')
def grow(x):
  return x + x

assert grow(3) == 6
print(f'Output: {grow(3)}')

ERROR:loial.builders.python_builder:Error compiling code, using defaut: invalid syntax (<string>, line 1)


Output: 6
